# Fase 3: Evaluación de Diferencias en Reservas de Vuelos por Nivel Educativo


1. Preparación de Datos: 

    - Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

2. Análisis Descriptivo: 

    - Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

3. Prueba Estadística:

    - Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

### Importamos las librerías que necesitamos

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
from src import support as sp

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Creación de muestras 
# -----------------------------------------------------------------------
from sklearn.model_selection import train_test_split

In [2]:
# Cargamos los datos generados en la Fase 1
df_combined = pd.read_csv('../data/processed/df_combined.csv', index_col=0)

In [3]:
# Mostramos las primeras filas del DataFrame
df_combined.head(3) 

,loyalty_number,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.000000,Married,Aurora,7919.20,Standard,2016,8,NaN,NaN
1,100102,2017,1,10,4,14,2030,203.0,0,0,Ontario,Toronto,M1R 4K3,Male,College,64778.033656,Single,Nova,2887.74,Standard,2013,3,NaN,NaN
2,100140,2017,1,6,0,6,1200,120.0,0,0,British Columbia,Dawson Creek,U5I 4F1,Female,College,64476.508588,Divorced,Nova,2838.07,Standard,2016,7,NaN,NaN


- Filtra el conjunto de datos para incluir únicamente las columnas relevantes: 'Flights Booked' y 'Education'.

In [4]:
# Filtrar el conjunto de datos para incluir las columnas 'flights_booked' y 'education'
df_filtered = df_combined[['flights_booked', 'education']]
df_filtered.shape

(401688, 2)

* Si solo filtramos por esas 2 columnas, luego perdemos la referencia de quienes son estos clientes, por lo cuál, filtraremos por 3 columnas

In [5]:
# Filtrar el conjunto de datos para incluir las columnas 'loyalty_number', 'flights_booked' y 'education'
df_filtered = df_combined[['loyalty_number', 'flights_booked', 'education']]
df_filtered.shape

(401688, 3)

- Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estandar, los percentiles) del número de vuelos reservados para cada grupo.

In [6]:
df_filtered.groupby('education')['flights_booked'].describe()

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
Bachelor,251400.0,4.090354,5.214194,0.0,0.0,1.0,8.0,21.0
College,101712.0,4.145656,5.237434,0.0,0.0,1.0,8.0,21.0
Doctor,17616.0,4.149296,5.246376,0.0,0.0,1.0,8.0,21.0
High School or Below,18768.0,4.142210,5.229254,0.0,0.0,1.0,8.0,21.0
Master,12192.0,4.167815,5.204100,0.0,0.0,1.0,8.0,21.0


- Realiza una prueba de A/B testing para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

Lo primero que debemos hacer en un A/B testing es verificar que los datos son coherentes y están listos. En este caso, ya tenemos este paso concluído.

Dado que cada cliente tiene 24 entradas (una por cada mes en dos años), vamos a calcular el promedio mensual de vuelos reservados por cada cliente. Así tendremos una sola fila por cliente con la media de vuelos reservados.

In [7]:
# Agrupamos por loyalty_number y education y calculamos la media de vuelos reservados por cada cliente
df_filtered_ab = df_filtered.groupby(['loyalty_number', 'education'])['flights_booked'].mean().reset_index() 
df_filtered_ab.tail()

,loyalty_number,education,flights_booked
16732,999902,College,7.416667
16733,999911,Doctor,0.000000
16734,999940,Bachelor,2.625000
16735,999982,College,0.916667
16736,999986,Bachelor,5.041667


### Ahora vamos a crear nuestro test_group, categorizando la columna education en 2 grupos: 
    * GRUPO A: Clientes con nivel de educación inferior: High School or Below y College
    * GRUPO B: Clientes con nivel de educación superior: Bachelor, Doctor y Master

Recordemos el orden de los niveles de educación de menor a mayor:

1. **High School or Below** (Secundaria o menos)
2. **College** (Universidad, generalmente implica que no se completó un título universitario)
3. **Bachelor** (Licenciatura)
4. **Master** (Máster)
5. **Doctor** (Doctorado)

In [8]:
df_filtered_ab.education.value_counts() # Miramos como están distribuidos los niveles de educación

education
Bachelor                10475
College                  4238
High School or Below      782
Doctor                    734
Master                    508
Name: count, dtype: int64

In [9]:
# Usamos el apply para aplicar nuestra función que categoriza los distintos niveles de education
group_a = ['High School or Below', 'College']
group_b = ['Bachelor', 'Master', 'Doctor']

df_filtered_ab['test_group'] = df_filtered_ab['education'].apply(lambda x: sp.categorize(x, group_a, group_b))

In [10]:
df_filtered_ab.sample(5)

,loyalty_number,education,flights_booked,test_group
11314,707592,Master,2.166667,group_b
13918,845782,Bachelor,4.041667,group_b
16125,967017,Bachelor,8.916667,group_b
10672,672917,Bachelor,0.000000,group_b
4949,367961,College,6.291667,group_a


### Planteamos las hipótesis

- Hipótesis nula (H0): No existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.
- Hipótesis alternativa (H1): Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.

### Definimos si es un problema de medias o de proporciones

In [11]:
pd.crosstab(df_filtered_ab.flights_booked, df_filtered_ab.test_group).shape # Con una crosstab confirmamos que es un problema de médias, nuestra crosstab tiene 225 filas.

(225, 2)

In [12]:
sp.determine_problem_type(df_filtered_ab, 'flights_booked') # Tenemos una función que también nos hace esta comprobación 

'means'

### Aplicamos una prueba de normalidad a nuestra métrica

In [13]:
sp.normality_test(df_filtered_ab, 'flights_booked', method='ks')

The data for the metric 'flights_booked' does not follow a normal distribution (p-value = 0.0000).


(0.762883276127915, 0.0)

Concluimos que los datos no siguen una distribución normal, por lo tanto usaremos una prueba no paramétrica

In [14]:
sp.mann_whitney_test(df_filtered_ab, ['flights_booked'], 'group_a', 'group_b')

For the metric 'flights_booked', the medians are the same (p-value = 0.3117).


### Interpretación de Resultados de la Prueba de Mann-Whitney U

**Métrica Evaluada:** `flights_booked`

**Resultado de la Prueba:**
- **p-value:** 0.3117

**Hipótesis:**
- **Hipótesis nula (H0):** No existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.
- **Hipótesis alternativa (H1):** Existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B.

**Interpretación:**

El resultado de la prueba de Mann-Whitney U indica un p-value de 0.3117. Dado que este valor es considerablemente mayor que el nivel de significancia comúnmente aceptado de 0.05, no rechazamos la hipótesis nula.

**Conclusión:**

Con un p-value de 0.3117, no encontramos evidencia suficiente para afirmar que existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos de los grupos A y B. Por lo tanto, según los datos analizados, no hay una diferencia significativa en el número de vuelos reservados entre estos grupos. Esto sugiere que el nivel educativo no tiene un impacto significativo en la cantidad de vuelos reservados por los clientes en los grupos comparados.

---

Podríamos seguir profundizando este análisis y realizando otros A/B testing cambiando las categorias educativas que pertenecen a los grupos A y B, o incluso podríamos realizar pruebas con otras métricas. 